In [ ]:
# This script is used to find images which have the most different prediction in 2 experiments
# input: the prediction folder of 2 experiments
# output: list of images which have the most different predicted masks

In [20]:
import numpy as np
import os
from PIL import Image
# cls_path = "/data2/wjl/diffusion/ddsseg/data/voc/cls_labels.npy"
cls_path = "/data2/wjl/diffusion/ddsseg/data/synthetic/cls_labels.npy"
labels = np.load(cls_path, allow_pickle=True)
# ids_path = "/data2/wjl/diffusion/ddsseg/data/voc/train_id.txt"
ids_path = "/data2/wjl/diffusion/ddsseg/data/synthetic/random_category.txt"
df = open(ids_path)
name_list = [name.strip() for name in df.readlines()]
# data_root = "/data2/wjl/diffusion/ddsseg/datasets/VOCdevkit/VOC2012/SegmentationClassAug"
mask_root = "/data2/wjl/diffusion/ddsseg/datasets/synthetic_dataset/dataset/masks"
sds_threshold = 0.43
dds_threshold = 0.43

In [21]:
cls2idx = {
  'plane': 1,
  'bicycle': 2,
  'bird':3,
  'boat': 4,
  'bottle':5,
  'buses': 6,
  'car':7,
  'cat':8,
  'chair':9,
  'cow': 10,
  'table':11 ,
  'dog': 12,
  'horse': 13,
  'motorbike':14 ,
  'people':15,
  'plant': 16,
  'sheep': 17,
  'sofa': 18,
  'train': 19,
  'monitor': 20}
idx2cls = ['plane', 'bicycle', 'bird', 'boat', 'bottle', 'buses', 'car', 'cat', 'chair', 'cow', 
           'table', 'dog', 'horse', 'motorbike', 'people', 'plant', 'sheep', 'sofa', 'train', 'monitor']

In [22]:
def calculate_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    iou = np.sum(intersection) / np.sum(union)
    iou = int(iou * 100) / 100
    return iou

def count_iou(id,name_list, img_path, mask_root, threshold):
    name = name_list[id]
    gt_path = os.path.join(mask_root, name + ".png")
    gt =  np.array(Image.open(gt_path))
    gt[gt==255]=0
    idx_list = np.unique(gt)[1:]
    gt_mask = [gt == 0]
    for idx in idx_list:
        mask = (gt == idx)
        gt_mask.append(mask)
    
    pre_list = []
    for idx in idx_list:
        cls = idx2cls[idx-1]
        predict_img_path = os.path.join(img_path, str(id) + "_" + cls + ".png")
        predict_img = np.array(Image.open(predict_img_path))
        predict_img = (predict_img - predict_img.min())/(predict_img.max() - predict_img.min())
        pre_list.append(predict_img)
        
    dds_array = np.full(pre_list[0].shape, threshold)
    pre_list = [dds_array] + pre_list
    pre_list = np.array(pre_list)
    
    mask_pred = np.argmax(pre_list, axis=0)
    iou_dict = {}
    fg_iou = 0
    for i in range(len(pre_list)):
        iou = calculate_iou(mask_pred == i, gt_mask[i])
        if i==0:
            iou_dict['bg'] = iou
        else:
            iou_dict[idx2cls[idx_list[i-1]-1]] = iou
            fg_iou += iou
    fg_iou /= len(idx_list)
    return iou_dict, fg_iou



In [ ]:
dds_path = "/data2/wjl/diffusion/ddsseg/output/synthetic/dds_baseline/masks"
sds_path = "/data2/wjl/diffusion/ddsseg/output/synthetic/sds_baseline/masks"
# dds_dict, fg_dds = count_iou(229,name_list, dds_path, mask_root, dds_threshold)
cmp_list = []
dds_dict_list = []
sds_dict_list = []
for i in range(len(name_list)):
    dds_dict, fg_dds = count_iou(i,name_list, dds_path, mask_root, dds_threshold)
    sds_dict, fg_sds = count_iou(i,name_list, sds_path, mask_root, sds_threshold)
    cmp_list.append(fg_dds-fg_sds)
    dds_dict_list.append(dds_dict)
    sds_dict_list.append(sds_dict)

In [ ]:
sorted_indices = sorted(range(len(cmp_list)), key=lambda i: cmp_list[i], reverse=True)
top_20_indices = sorted_indices[:20]
for indice in top_20_indices:
    print(indice)
    print(f"dds:{dds_dict_list[indice]}")
    print(f"sds:{sds_dict_list[indice]}")
    print(cmp_list[indice])
    print("\n")